In [2]:
pip install --upgrade google-cloud-discoveryengine humanize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Enable APIs
 In this task, you need to enable the Dialogflow API.

 To enable the Dialogflow API, follow these steps:

 Type Dialogflow API into the top search bar of the Google Cloud console, choose the selected result as shown below, and continue. It enables the conservation flow interfaces like chats, video 
 
 
 Install prerequisites
In the Google Cloud console, navigate to Vertex AI Workbench. In the top search bar of the Google Cloud console, enter Vertex AI Workbench, and click on the first result.

In [1]:
import IPython


In [4]:
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# Helper function
1.Run the following code in the next cell to setup the helper methods.

In [1]:
import humanize
import time
import re
from typing import List, Optional

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine


def _call_list_documents(
    project_id: str, location: str, datastore_id: str, page_token: Optional[str] = None
) -> discoveryengine.ListDocumentsResponse:
    """Build the List Docs Request payload."""
    client_options = (
        ClientOptions(
            api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(
        client_options=client_options)

    request = discoveryengine.ListDocumentsRequest(
        parent=client.branch_path(
            project_id, location, datastore_id, "default_branch"
        ),
        page_size=1000,
        page_token=page_token,
    )

    return client.list_documents(request=request)


def list_documents(
    project_id: str, location: str, datastore_id: str, rate_limit: int = 1
) -> List[discoveryengine.Document]:
    """Gets a list of docs in a datastore."""

    res = _call_list_documents(project_id, location, datastore_id)

    # setup the list with the first batch of docs
    docs = res.documents

    while res.next_page_token:
        # implement a rate_limit to prevent quota exhaustion
        time.sleep(rate_limit)

        res = _call_list_documents(
            project_id, location, datastore_id, res.next_page_token
        )
        docs.extend(res.documents)

    return docs


def list_indexed_urls(
    docs: Optional[List[discoveryengine.Document]] = None,
    project_id: str = None,
    location: str = None,
    datastore_id: str = None,
) -> List[str]:
    """Get the list of docs in data store, then parse to only urls."""
    if not docs:
        docs = list_documents(project_id, location, datastore_id)
    urls = [doc.content.uri for doc in docs]

    return urls


def search_url(urls: List[str], url: str) -> None:
    """Searches a url in a list of urls."""
    for item in urls:
        if url in item:
            print(item)


def search_doc_id(
    doc_id: str,
    docs: Optional[List[discoveryengine.Document]] = None,
    project_id: str = None,
    location: str = None,
    datastore_id: str = None,
) -> None:
    """Searches a doc_id in a list of docs."""
    if not docs:
        docs = list_documents(project_id, location, datastore_id)

    doc_found = False
    for doc in docs:
        if doc.parent_document_id == doc_id:
            doc_found = True
            print(doc)

    if not doc_found:
        print(f"Document not found for provided Doc ID: `{doc_id}`")


def estimate_data_store_size(
    urls: Optional[List[str]] = None,
    docs: Optional[List[discoveryengine.Document]] = None,
    project_id: str = None,
    location: str = None,
    datastore_id: str = None,
) -> None:
    """For Advanced Website Indexing data stores only."""
    if not urls:
        if not docs:
            docs = list_documents(project_id, location, datastore_id)
        urls = list_indexed_urls(docs=docs)

    # Filter to only include website urls.
    urls = list(filter(lambda x: re.search(r"https?://", x), urls))

    if not urls:
        print(
            "No urls found. Make sure this data store is for websites with advanced indexing."
        )
        return

    # For website indexing, each page is calculated as 500KB.
    size = len(urls) * 500_000
    print(f"Estimated data store size: {humanize.naturalsize(size)}")


PENDING_MESSAGE = """
No docs found.\n\nIt\'s likely one of the following issues: \n  [1] Your data store is not finished indexing. \n  [2] Your data store failed indexing. \n  [3] Your data store is for website data without advanced indexing.\n\n
If you just added your data store, it can take up to 4 hours before it will become available.
"""

# User Inputs
Creating a new data store.
1. Navigate to Vertex AI Search & Conversation console and click on CONTINUE AND ACTIVATE THE API button.
Vertex AI Search and Conversation allows developers to quickly build new experiences such as custom search engines and conversational apps via out-of-the-box templates and APIs.
2. In the Create App page, select Chat as an App type.

3. For Company Name enter Cymbal. For Agent Name enter cymbalagent and click CONTINUE.

4. In the Data Stores page, click + CREATE NEW DATA STORE.

5. Select Cloud Storage, and enter the following Google Cloud Storage location cloud-samples-data/dialogflow-cx/arc-lifeblood to add the folder. Then select Unstructured documents under What kind of data are you importing? section and finally click CONTINUE.
6. For Data store name, enter cymbaldatastore and click CREATE. This creates a Data store.

7. Finally on the App's Data page, select cymbaldatastore and click on CREATE.
8. Click on cymbaldatastore and note down the Data store ID.
9. Alternatively, you can find the datastore_id by from Navigation Menu navigation-menu > Search & Conversation.

10. For cymbalagent, click on View under Connected data stores.
11. Finally, click on cymbaldatastore to view the Data store ID. Note down the Data store ID.

Data store ID
cymbalagent_1710444207891



# Task 5. Check Data Store Index Status
Let's use the list_documents method, to check if the data store has finished indexing.

Back in the Jupyter notebook, run the following command in the next cell and replace the YOUR_DATA_STORE_ID with the Data store ID noted down in earlier task.

In [7]:
PROJECT = !gcloud config get-value project
project_id  = PROJECT[0]
print(project_id)
location = "global"  # Options: "global", "us", "eu"
datastore_id = "cymbaldatastore_1710444109310"

qwiklabs-gcp-00-40732e0c8212


In [8]:
docs = list_documents(project_id, location, datastore_id)

if len(docs) == 0:
    print(PENDING_MESSAGE)
else:
    SUCCESS_MESSAGE = f"""
  Success! 🎉\n
  Your indexing is complete.\n
  Your index contains {len(docs)} documents.
  """
    print(SUCCESS_MESSAGE)


  Success! 🎉

  Your indexing is complete.

  Your index contains 79 documents.
  


In [9]:
docs = list_documents(project_id, location, datastore_id)
docs[0]

name: "projects/798934074668/locations/global/collections/default_collection/dataStores/cymbaldatastore_1710444109310/branches/0/documents/01c3140622cfed9a86572e550ed049a0"
id: "01c3140622cfed9a86572e550ed049a0"
schema_id: "default_schema"
struct_data {
}
parent_document_id: "01c3140622cfed9a86572e550ed049a0"
content {
  mime_type: "text/html"
  uri: "gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/testing.html"
}

# Task7. Search Data Store by Doc ID
Let's search through all docs in a given Data Store and find a specific Doc ID.

In the following command, replace the placeholder_document_id with the value of parent_document_id from the last output.

In [11]:
document_id = "01c3140622cfed9a86572e550ed049a0"
search_doc_id(document_id, docs=docs)

name: "projects/798934074668/locations/global/collections/default_collection/dataStores/cymbaldatastore_1710444109310/branches/0/documents/01c3140622cfed9a86572e550ed049a0"
id: "01c3140622cfed9a86572e550ed049a0"
schema_id: "default_schema"
struct_data {
}
parent_document_id: "01c3140622cfed9a86572e550ed049a0"
content {
  mime_type: "text/html"
  uri: "gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/testing.html"
}



# Task 8. List Indexed URLs
Run the following code snippet in the next notebook cell, to retrieve a indexed URL.

In [12]:
urls = list_indexed_urls(docs=docs)
urls[0]

'gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/testing.html'

# Task 9. Search Indexed URLs
Run the following code snippet in the next notebook cell, to search for a specific URL within a list of indexed URLs.

In [14]:
search_url(urls, "gs://cloud-samples-data/dialogflow-cx/arc-lifeblood")

gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/testing.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/strategy.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/making-blood-components.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/making-your-donation.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/products.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/high-ferritin.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/forms.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/blood-for-transfusion.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/how-you-can-give-life.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/transplantation-immunogenetics-services.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/blood.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/prepare-and-aftercare.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/our-strategy.html
gs://cloud-samples-data/dialogflow-cx/arc-l

In [15]:
search_url(urls, "dialogflow-cx")

gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/testing.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/strategy.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/making-blood-components.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/making-your-donation.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/products.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/high-ferritin.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/forms.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/blood-for-transfusion.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/how-you-can-give-life.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/transplantation-immunogenetics-services.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/blood.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/prepare-and-aftercare.html
gs://cloud-samples-data/dialogflow-cx/arc-lifeblood/our-strategy.html
gs://cloud-samples-data/dialogflow-cx/arc-l